In [1]:
library(tidyverse)
library(sva)

source("../../utils/plots_eda.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: mgcv

Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:dplyr’:

    collapse


This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Loading required package: genefilter


Attaching package: ‘genefilter’


The following object is masked from ‘package:readr’:

    spec


Loading required package: BiocParallel


Attaching package: ‘gridExtra’


The following object is masked from ‘pa

# Load data

In [2]:
datasets <- c("GSE6008", "GSE14407", "GSE26712",  "GSE40595", "GSE36668", "GSE69428")

In [3]:
all_metadata <- read.table("before/all_design.tsv", header = TRUE, sep = "\t")
all_expression <- read.table("before/all_expr_for_correction.tsv", header = TRUE, sep = "\t") %>%
    column_to_rownames("gene_id")

all_expression <- all_expression[, all_metadata$sample_id]

# remove NAs rows
all_expression <- na.omit(all_expression)
# remove genes with 0 variance
all_expression <- all_expression[apply(all_expression, 1, var) > 0, ]

In [5]:
print("Loading metadata and expression data")
print("Metadata dimensions:")
print(dim(all_metadata))
print("Expression data dimensions:")
print(dim(all_expression))

[1] "Loading metadata and expression data"
[1] "Metadata dimensions:"
[1] 309   3
[1] "Expression data dimensions:"
[1] 13237   309


In [6]:
design <- model.matrix(~all_metadata$Status)

corrected_expr <- sva::ComBat(dat = all_expression, 
                              batch = all_metadata$batch, 
                              mod = design)

corrected_expr <- as.data.frame(corrected_expr)


Found6batches

Adjusting for1covariate(s) or covariate level(s)

Standardizing Data across genes

Fitting L/S model and finding priors

Finding parametric adjustments

Adjusting the Data




In [7]:
all_metadata$Status <- all_metadata$Status
all_metadata$Dataset <- as.character(all_metadata$batch)

In [8]:

# plot the combined corrected data
print("Plotting combined corrected data")
plot_res <- plot_diagnostic(corrected_expr, all_metadata, "Combined Corrected",
                            log_transform = TRUE, with_rowname = TRUE)
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
          (plot_res[[3]] )
ggsave("after/diagnostic_plot_corrected.png", 
            plot = layout, width = 12, height = 12)


[1] "Plotting combined corrected data"
[1] "..plotting.."


Warning message:
“`aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.”
No id variables; using all as measure variables



In [9]:
# write out the corrected expression data
write.table(corrected_expr %>% rownames_to_column("gene_id"), "after/all_corrected_R_expr.tsv", sep = "\t", 
            quote = FALSE, col.names = TRUE, row.names = FALSE)